When I first heard about data science, I did not really sure where should I start and what the path is to become a Data Scientist. After countless hours of research my conclusion about what the job consists of was unclear. What I had discovered in my research was how this job title is rapidly growing into a demanding career option. Living in a time of enormous amounts of data, more and more companies are in demand and looking to fill this position. I began to question, how do you become a data scientist, what degree or level of education do you need, what skills are employers looking for? This lead me to explore the job applications, backgrounds, experiences, and skills that the current data scientist possess. So the data analysis begins!
first, I scrape LinkedIn job and I supposed with this I would gather job descriptions and key skills demanding. Second I am going to scrape individuals who land the job as a Data Scientist.
For the first part I will go with getting insights from job descriptions. Let ‘s start scraping!


In [1]:
# selenium
from selenium.webdriver.common.by import By
import selenium.webdriver as webdriver
import selenium.webdriver.support.ui as ui
from selenium.webdriver.common.keys import Keys
from selenium import webdriver
from selenium.webdriver.common.by import By
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC

#sys libraries
from time import sleep
import requests
import random
import csv
import io
import os
import sys
import types
import getpass
import string
import re
from bs4 import BeautifulSoup

# Import data viz libraries
import matplotlib.pyplot as plt
import seaborn as sns
import pandas as pd
import numpy as np

# nltk and wordcloud
import nltk
from nltk.corpus import stopwords
from wordcloud import WordCloud

%matplotlib inline

pd.set_option('display.max_colwidth', 100)
pd.set_option('display.max_rows' , 100)

### The Web-scrape:
I had to decide on what country and job title to pull job information from. I decided to go with United Kingdom due to the enormous amounts of applications for data scientists and the English language standard libraries for analyzing text data. 

>First, I do not condone scraping LinkedIn data in any way. Anyone who wishes to do so should first read <a href = 'https://www.linkedin.com/help/linkedin/answer/56347/prohibition-of-scraping-software?lang=enprohibition'>  LinkedIn's statement </a> on their of scraping software. 

>Also, note that LinkedIn actively suspends accounts due to <b>excessive activity</b> on a given account.

This code is to be used for learning purposes only and should not be used a tool for any commercial purpose. 

I used selenium and beautiful soup to web-scrape job application in United Kingdom in LinkedIn. 
When searching through the list of job application on LinkedIn, it will show a number of pages with 25 job applications on each page. After your first page, to continue scraping on to the next set of 25 job applications you have to get to the next page. Using selenium helped me maneuver around this issue. Selenium has a restriction on its speed because the scraping with the browser is much slower. Due to the slowness, to not get banned by LinkedIn the use of the "sleep" statements had to be used in my code multiple times to cause further slow-down. I write every job applicant to a csv file.





#### linkedin account config

In [2]:
# getting from user input
# try:
#     username = input('Enter your Username ')
#     password = getpass.getpass('Enter your password ')
#     desired_job = input('Searched job query: ')
#     desired_location = input('Location: ')
# except Exception as error: 
#     print('ERROR', error) 
    
    
username = 'behdad.k7@gmail.com'
password = '************'
desired_keyword = 'data scientist in united kingdom'
desired_location = 'United Kingdom'

csv_file = str.strip(desired_keyword)
file_name = csv_file + "_" + str(desired_location) +'.csv'

#### Scraping Linkedin jobs and save in a CSV file

In [3]:
driver = webdriver.Firefox(executable_path= "geckodriver.exe")

profile = webdriver.FirefoxProfile()
profile.set_preference("browser.cache.disk.enable", False)
profile.set_preference("browser.cache.memory.enable", False)
profile.set_preference("browser.cache.offline.enable", False)
profile.set_preference("network.http.use-cache", False)
profile.update_preferences()

driver.get(url='https://www.linkedin.com/')
sleep(2)

# Find the sign in botthon
sign_in = driver.find_element(by='link text', value='Sign in')
sign_in.click()
sleep(5)

# We find the input section and enter our user information.
email_entry = driver.find_element(by='css selector', value='#username')
password_entry = driver.find_element(by='css selector', value='#password')

# For every letter in our email, we will set a random time between keystrokes
for letter in username:
    sleep(random.uniform(.1, .4))
    email_entry.send_keys(letter)

# We expect a random time between 0.1 and 0.4 sec for each letter of our password.
for letter in password:
    sleep(random.uniform(.1, .4))
    password_entry.send_keys(letter)

# We press enter
password_entry.send_keys(Keys.RETURN)
sleep(10)

# click on typehead search bar
search_bar = WebDriverWait(driver, 20).until(
    EC.presence_of_element_located((By.XPATH, "//input[@placeholder='Search']")))
search_bar.click()

# Enter the job search term and press enter
for letter in desired_keyword:
    sleep(random.uniform(.1, .4))
    search_bar.send_keys(letter)
search_bar.send_keys(Keys.RETURN)
sleep(3)


# Find and Click on just people button
just_people_btn = WebDriverWait(driver, 10).until(
    EC.presence_of_element_located((By.XPATH, "//button[@aria-label='View only People results']")))
just_people_btn.click()
sleep(2)

source = requests.get(driver.current_url).text
soup = BeautifulSoup(source, 'lxml')

j = 1

# open the csv file to wirte
writer = csv.writer(open(file_name, 'w', encoding='utf-8'))
#write the headers
writer.writerow(['name', 'job_title', 'current_company', 'university','degree', 'location', 'about', 'skills'
                        'present_url'])

main_url = driver.current_url
present_url = main_url
while (j < 100):

#     print(present_url)
    driver.get(present_url)
    sleep(10)
    
    driver.execute_script("window.scrollTo(0, document.body.scrollHeight);")
    sleep(1)
    
    links = driver.find_elements_by_xpath('//div[contains(@class, "search-result__info")]//a[@href]')
    profiles_link = set()
    for link in links:
        link_add=link.get_attribute('href')
        print(link_add)
        if link_add.startswith('https://www.linkedin.com/in/'):
            profiles_link.add(link_add)
            
    for profile in profiles_link:
        try:
            driver.get(profile)
            sleep(5)
        except:
            pass
        
        # -------------Scrap here----------------
            
        print('---------------------------')
        print(profile)
        driver.execute_script("window.scrollTo(0, document.body.scrollHeight);")
        sleep(.5)
        

        ''' 1)Name'''
        try:
            name = WebDriverWait(driver, 10).until(
                EC.presence_of_element_located((
                    By.XPATH,'//li[contains(@class, "inline t-24 t-black t-normal break-words")]')))
            name = name.text
            if name:
                name = name.strip()
        except:
            name = np.nan
                
        print('name:\n',name , '\n')
        sleep(0.5)
        
        ''' 2)job_title'''
        try:
            job_title = WebDriverWait(driver, 10).until(
                EC.presence_of_element_located((
                    By.XPATH,'//h2[contains(@class, "mt1 t-18 t-black t-normal")]')))
            job_title = job_title.text
            if job_title:
                job_title = job_title.strip()
        except:
            job_title = np.nan
                
        print('job_title:\n',job_title , '\n')
        sleep(0.5)
        
        ''' 3)current_company'''
        try:
            current_company = WebDriverWait(driver, 10).until(
                EC.presence_of_element_located((
                    By.XPATH,'//ul[contains(@class, "pv-top-card--experience-list")]/li[1]')))
            current_company = current_company.text
            if current_company:
                current_company = current_company.strip()
        except:
            current_company = np.nan
                
        print('current_company:\n',current_company , '\n')
        sleep(0.5)
        
        ''' 4)university'''
        try:
            university = WebDriverWait(driver, 15).until(
                EC.presence_of_element_located((
                    By.XPATH,'//ul[contains(@class, "pv-top-card--experience-list")]/li[2]')))
            if university =='':
                degree = WebDriverWait(driver, 10).until(
                    EC.presence_of_element_located((
                        By.XPATH,'//section[contains(@class, "pv-profile-section education-section ember-view")]')))
            degree = WebDriverWait(driver, 10).until(
                EC.presence_of_element_located((
                    By.XPATH,'//li[1]//div[contains(@class,"pv-entity__degree-info")]')))
            degree = WebDriverWait(driver, 10).until(
                EC.presence_of_element_located((
                    By.XPATH,'//h3[contains(@class,"pv-entity__school-name t-16 t-black t-bold")]')))
            
            university = university.text
            if university:
                university = university.strip()
        except:
            university = np.nan
                

        print('university:\n',university , '\n')
        sleep(0.5)
        
        
        ''' 5)degree'''
        try:
            degree = WebDriverWait(driver, 10).until(
                EC.presence_of_element_located((
                    By.XPATH,'//section[contains(@class, "pv-profile-section education-section ember-view")]')))
            degree = WebDriverWait(driver, 10).until(
                EC.presence_of_element_located((
                    By.XPATH,'//li[1]//div[contains(@class,"pv-entity__degree-info")]')))
            degree = WebDriverWait(driver, 10).until(
                EC.presence_of_element_located((
                    By.XPATH,'//span[contains(@class,"pv-entity__comma-item")]')))
            degree = degree.text
            if degree:
                degree = degree.strip()
        except:
            degree = np.nan
                
        print('degree:\n',degree , '\n')
        sleep(0.5)
        
        ''' 6)location'''
        try:
            location = WebDriverWait(driver, 10).until(
                EC.presence_of_element_located((
                    By.XPATH,'//ul[contains(@class, "pv-top-card--list pv-top-card--list-bullet mt1")]/li[1]')))
            location = location.text
            if location:
                location = location.strip()
        except:
            location = np.nan
        
        print('location:\n',location , '\n')
        sleep(0.5)
        
        ''' 7)about'''
        try:
            about = WebDriverWait(driver, 20).until(
                EC.presence_of_element_located((
                    By.XPATH,'//section[contains(@class, "artdeco-container-card pv-profile-section pv-about-section ember-view")]//a[contains(@class,"lt-line-clamp__more")]'))).click()
            about = WebDriverWait(driver, 20).until(
                EC.presence_of_element_located((
                    By.XPATH,'//p[contains(@class,"pv-about__summary-text mt4 t-14 ember-view")]')))      
            about = about.text
            if about:
                about = about.strip()
        except:
            about = np.nan
             
        print('about:\n',about , '\n')
        sleep(0.5)
        
        ''' 8)skills'''
        
        try:
            print("skills search ... ")
            skills = WebDriverWait(driver, 20).until(
                EC.presence_of_element_located((
                    By.XPATH,'//li[contains(@class , "pv-skill-category-entity__top-skill pv-skill-category-entity pb3 pt4 pv-skill-endorsedSkill-entity relative ember-view")]//span[contains(@class, "pv-skill-category-entity__name-text t-16 t-black t-bold")]')))
            
#             skill = driver.find_elements_by_class_name('pv-skill-category-entity__name-text t-16 t-black t-bold')
            skills = skills.text

#             skills = [skill.text for skill in skills]
#             skills = ','.join(skills)
            if skills:
                skills = skills.strip()
        
        except:
            skills = np.nan
                
        print('skills:\n',skills , '\n')
        sleep(0.5)
        
        
        
        writer.writerow([ name, job_title, current_company, university, degree, location, about
                        ,skills, profile])
            
        sleep(2)
        
        
    
    present_url = main_url + '&page=' + str(j)
    j = j + 1

print("The process is completed .../n closing The program!")
driver.quit()


https://www.linkedin.com/in/rahul-jaiswal-570042ab/
https://www.linkedin.com/in/sasan-raeissian-b4b1a034/
https://www.linkedin.com/in/hesam-z-ghoochaninejad/
https://www.linkedin.com/search/results/people/?facetConnectionOf=%5B%22ACoAABdSBKQB0lfwpj9fMD0pHHa9ywhYaNlfdUQ%22%5D&facetNetwork=%5B%22F%22%5D&origin=SHARED_CONNECTIONS_CANNED_SEARCH
https://www.linkedin.com/in/andriasarri/
https://www.linkedin.com/in/ecaterina-vidrascu/
https://www.linkedin.com/in/salim-namvar/
https://www.linkedin.com/in/hassan-heidari-fathian/
https://www.linkedin.com/search/results/people/?facetConnectionOf=%5B%22ACoAABbTvrEB_OTqW0ubJEs6-mnJ7nkvqs8lRyY%22%5D&facetNetwork=%5B%22F%22%5D&origin=SHARED_CONNECTIONS_CANNED_SEARCH
https://www.linkedin.com/in/mahdi-ghanbari-4988ab77/
https://www.linkedin.com/in/farzaneh-termeh-818a783b/
https://www.linkedin.com/in/mhbashari/
https://www.linkedin.com/search/results/people/?facetConnectionOf=%5B%22ACoAABBUBvQBpBcwg9rBCqhZ2ySWgGkv4u7Sh_I%22%5D&facetNetwork=%5B%22F%22%5

skills search ... 
skills:
  

---------------------------
https://www.linkedin.com/in/hesam-z-ghoochaninejad/
name:
 Hesam Z. Ghoochaninejad 

job_title:
 Petrophysicist/Data Scientist 

current_company:
 Iranian Offshore Oil Company 

university:
 nan 

degree:
 nan 

location:
 Iran 

about:
 Experienced in reservoir's Petrophysics, 3D Modeling, and Geosteering.
Utilizing Data Mining and Machine Learning techniques in the Upstream Industry.
Developing my expertise in data science and emerging technologies. 

skills search ... 
skills:
 nan 

---------------------------
https://www.linkedin.com/in/mohammad-amin-khodamoradi-115705167/
name:
 Mohammad Amin khodamoradi 

job_title:
 Data Scientist at Laklak 

current_company:
 Laklak 

university:
 Shahid Beheshti University 

degree:
 Master's degree 

location:
 Iran 

about:
 I am an M.sc. of Computer Science from Shahid Beheshti University, Iran. My research and thesis focused on Data Science and Bioinformatics. Also, I'm interested

name:
 Subbarao Y 

job_title:
 Artificial Intelligence machine learning - Chatbot developer 

current_company:
 DBS Bank 

university:
 nan 

degree:
 nan 

location:
 Nellore, Andhra Pradesh, India 

about:
 *IT Professional with 11 Years of Experience in Software industry.
*good working knowledge on Artificial intelligence machine learning &RASA,IBM Watson chatbot applications.
*Good working knowledge on facial recognition.
*Good knowledge on Microsoft Azur chatbot application.
*Excellent working knowledge on Kony studio,KMF, Visualizer&Fabric8.x. 

skills search ... 
skills:
 nan 

---------------------------
https://www.linkedin.com/in/vaibhav9518s/
name:
 Vaibhav Sharma 

job_title:
 Data Scientist at ZappyAI 

current_company:
 ZappyAI 

university:
 London Business School 

degree:
 Masters in Analytics and Management 

location:
 London, Greater London, United Kingdom 

about:
 nan 

skills search ... 
skills:
  

---------------------------
https://www.linkedin.com/in/parasah

---------------------------
https://www.linkedin.com/in/professor-mo-saraee-90b0a11a/
name:
 Professor Mo Saraee 

job_title:
 Professor of Data Science at The University of Salford 

current_company:
 School of Science, Engineering & Environment, The University of Salford-Manchester 

university:
 nan 

degree:
 nan 

location:
 Manchester, Greater Manchester, United Kingdom 

about:
 nan 

skills search ... 
skills:
 nan 

---------------------------
https://www.linkedin.com/in/artemisnika/
name:
 Artemis Nika 

job_title:
 Data Scientist 

current_company:
 Farfetch 

university:
 University of Oxford 

degree:
 Master of Science (MSc) 

location:
 London, United Kingdom 

about:
 nan 

skills search ... 
skills:
  

---------------------------
https://www.linkedin.com/in/parasahuja/
name:
 Paras Ahuja 

job_title:
 Data Scientist at Jio 

current_company:
 Jio 

university:
 Indian Institute of Technology, Ropar 

degree:
 Engineer's Degree 

location:
 London, United Kingdom 

abo

current_company:
 Business Excellent Consultants Group (BCG) 

university:
 Allameh Tabataba'i University 

degree:
 Masters 

location:
 Iran 

about:
 nan 

skills search ... 
skills:
  

---------------------------
https://www.linkedin.com/in/subbarao-y-1547a8134/
name:
 Subbarao Y 

job_title:
 Artificial Intelligence machine learning - Chatbot developer 

current_company:
 DBS Bank 

university:
 nan 

degree:
 nan 

location:
 Nellore, Andhra Pradesh, India 

about:
 *IT Professional with 11 Years of Experience in Software industry.
*good working knowledge on Artificial intelligence machine learning &RASA,IBM Watson chatbot applications.
*Good working knowledge on facial recognition.
*Good knowledge on Microsoft Azur chatbot application.
*Excellent working knowledge on Kony studio,KMF, Visualizer&Fabric8.x. 

skills search ... 
skills:
 nan 

---------------------------
https://www.linkedin.com/in/dargaiton/
name:
 Stewart B. Dargaiton 

job_title:
 Data Scientist ● Data Analyst 

...

skills search ... 
skills:
  

---------------------------
https://www.linkedin.com/in/georgemissir/
name:
 George Missir 

job_title:
 Aspiring Data Scientist 

current_company:
 Todayware 

university:
 The London School of Economics and Political Science (LSE) 

degree:
 LSE Data Analysis for Management Programme 

location:
 Stratford-upon-Avon, Warwickshire, United Kingdom 

about:
 nan 

skills search ... 
skills:
  

---------------------------
https://www.linkedin.com/in/vivekkchandra/
name:
 Vivek Chandra 

job_title:
 Data scientist in training 

current_company:
 University of Essex Students' Union 

university:
 University of Essex 

degree:
 Master of Science - MS 

location:
 London, England Metropolitan Area 

about:
 nan 

skills search ... 
skills:
  

https://www.linkedin.com/in/brunasnobrega/
https://www.linkedin.com/in/jc-van-der-walt-49832a7b/
https://www.linkedin.com/in/maximo-corral-san-martin-2320204/
https://www.linkedin.com/in/delano-greenidge-013b857/
-------

skills search ... 
skills:
  

---------------------------
https://www.linkedin.com/in/sarah-miller-7026b8121/
name:
 Sarah Miller 

job_title:
 Data Scientist at KPMG 

current_company:
 KPMG 

university:
 Baruch College, City University of New York (CUNY) 

degree:
 Doctor of Philosophy (Ph.D.) 

location:
 London, Greater London, United Kingdom 

about:
 New American implant in London!

An established and professional Analyst and Data Scientist with a mathematics and statistics background who influences major business decisions by means of data science and strategic development. I am...

- Well-versed in the use of business intelligence and advanced analytics to help transform data into actionable insights

- Highly proficient programmer and statistical modeler, coding advanced quantitative analysis using statistical software (SAS, R), SQL, Python, and Excel;

- Experienced in bringing data to life through dashboard visualization software

- Adept in communicating and educating sup

skills search ... 
skills:
  

---------------------------
https://www.linkedin.com/in/peter-owlett-a015283b/
name:
 Peter Owlett 

job_title:
 Data Scientist at Thread 

current_company:
 Thread 

university:
 Durham University 

degree:
 Physics with Astronomy 

location:
 London, Greater London, United Kingdom 

about:
 Data scientist specialising in applied machine learning. My statistical focuses are regression, classification, time series forecasting and geospatial analytics.

I’ve worked across logistics, finance and consumer products and I’ve held back office, customer facing and leadership roles. I’m a passionate programmer and maker. I have a Masters in computational astrophysics from Durham University.

I love small teams, fast change and good pizza. 

skills search ... 
skills:
  

---------------------------
https://www.linkedin.com/in/chinmay-upadhyay-92804416/
name:
 Chinmay Upadhyay 

job_title:
 Data Scientist Principal | Advanced Analytics 

current_company:
 IHS Mark

skills search ... 
skills:
  

https://www.linkedin.com/in/barnaby-brien/
https://www.linkedin.com/in/mark-london-679b73a5/
https://www.linkedin.com/in/jordan-de-souza-5500333b/
https://www.linkedin.com/in/feifei-peng-b04a2986/
---------------------------
https://www.linkedin.com/in/barnaby-brien/
name:
 Barnaby Brien 

job_title:
 Data Scientist at Racing Post 

current_company:
 Spotlight Sports Group 

university:
 UCL 

degree:
 Master’s Degree 

location:
 London, Greater London, United Kingdom 

about:
 nan 

skills search ... 
skills:
  

---------------------------
https://www.linkedin.com/in/mark-london-679b73a5/
name:
 Mark London 

job_title:
 Data Scientist at GfK 

current_company:
 GfK 

university:
 Bournemouth University 

degree:
 Master's degree 

location:
 St John’s, Surrey, United Kingdom 

about:
 Data domain analysis
Creating strategy learning AI
Decisions from Wisdom, Information from Data

I believe that Data Science should be explainable - to at least one othe

skills search ... 
skills:
  

---------------------------
https://www.linkedin.com/in/naomi-hanlon-04a45374/
name:
 Naomi Hanlon 

job_title:
 Data Scientist at Sensum 

current_company:
 Sensum 

university:
 Queen's University Belfast 

degree:
 Master of Science (MSc) 

location:
 Belfast, United Kingdom 

about:
 nan 

skills search ... 
skills:
  

---------------------------
https://www.linkedin.com/in/qiaohongwang/
name:
 Qiaohong Wang 

job_title:
 Data Scientist at Facebook 

current_company:
 Facebook 

university:
 Peking University 

degree:
 B.Sc. 

location:
 London, Greater London, United Kingdom 

about:
 nan 

skills search ... 
skills:
  

---------------------------
https://www.linkedin.com/in/zhe-li-203227b9/
name:
 Zhe Li 

job_title:
 Data Scientist 

current_company:
 AstraZeneca 

university:
 UCL Faculty of Population Health Sciences 

degree:
 Master's degree 

location:
 United Kingdom 

about:
 nan 

skills search ... 
skills:
  

https://www.linkedin.com/i

In [4]:
df = pd.read_csv('data scientist in united kingdom_United Kingdom.csv',index_col=False)
df

,name,job_title,current_company,university,degree,location,about,skillspresent_url
0,Andria Sarri,Data Scientist,HSBC,UCL,Doctor of Philosophy (Ph.D.),"London, Greater London, United Kingdom",NaN,Data Analysis
1,Saksham Jindal,Data Scientist at Fractal Analytics,Fractal Analytics,"Indian Institute of Technology, Kharagpur",Dual Degree (Bachelor of Technology (B.Tech.) + Masters of Technology (M.Tech.)),"London, United Kingdom",NaN,C++
2,sasan raeissian,Senior Data Scientist at LGS - An IBM Company,LGS - An IBM Company,Université de Montréal,Master of Science (MSc),"Montreal, Canada Area",Summary of qualifications\r\n\r\nProfessionally qualified statistician with strong programming b...,NaN
3,Rahul Jaiswal,Data Scientist at HCL Technologies,HCL Technologies,Academy of Technology,BTech - Bachelor of Technology,"London, United Kingdom","Machine learning,Deep learning, Python,R ,NLP(Spacy,nltk,gensim), Statistical analysis.I have 3+...",NaN
4,Subbarao Y,Artificial Intelligence machine learning - Chatbot developer,DBS Bank,NaN,NaN,"Nellore, Andhra Pradesh, India",*IT Professional with 11 Years of Experience in Software industry.\r\n*good working knowledge on...,NaN
...,...,...,...,...,...,...,...,...
547,Taha Yasseri,"Computational Social Scientist, University of Oxford",The Alan Turing Institute,Georg-August-Universität Göttingen,Doctor of Philosophy (PhD),"Oxford, United Kingdom",Taha Yasseri is a Senior Research Fellow in Computational Social Science at the Oxford Internet ...,NaN
548,Hadi Sotudeh,Data Science Trainee at The Royal Dutch Football Association (KNVB),KNVB,NaN,NaN,"Eindhoven, North Brabant Province, Netherlands",NaN,NaN
549,Chris Parkinson,Data Scientist at Sky,Sky,Durham University,Master’s Degree,"Bexley, Greater London, United Kingdom","I am a technology professional with a background in mathematics, data science and cloud computin...",NaN
550,Matei Stefan Neagu,Data Scientist at Amazon,Amazon,The University of Sheffield,Doctor of Philosophy - PhD,United Kingdom,"I am deeply interested in data science and artificial intelligence, an intellectual curiosity wh...",NaN


## Exploratory Data Analysis (EDA)

In [ ]:
# Reading from scv file
df = pd.read_csv('data scientist_United Kingdom.csv',index_col=False)
df

In [ ]:
# find number of missing data based on each column
missing_data = df.isnull()
for column in missing_data.columns.values.tolist():
    print(column)
    print(missing_data[column].value_counts())
    print("\n")

### Do some cleaning and exploring on data
as you can see we have 775 rows × 11 columns. There some rows with null values. Because when we use selenium and getting elements with their class and atributes it has a chance that you got nothing!
We want to explore job applications description so let 's filter data by description

In [ ]:
jobs=df[df['job_description'].notnull()]
jobs

In [ ]:
jobs.columns

In [ ]:
# let 's drop skills and location columns
# because all the rows in skills are null and we scrape United Kingdom so we don't need location anymore!
del jobs['skill']
del jobs['location']

In [ ]:
jobs.dtypes

In [ ]:
jobs.head()

In [ ]:
#let 's see which company has the most applications for data science in united kingdom!
jobs['company'].value_counts().to_frame().reset_index()

In [ ]:
#let 's see what are the most title used for data science jobs in united kingdom!
print(len(jobs['job_title'].unique()) , ' Unique Title')
jobs['job_title'].value_counts().to_frame().reset_index()


In [ ]:
plt.figure(figsize=(10,5))
plt.tight_layout()

jobs['job_title'].value_counts().nlargest(15).plot(kind='bar')


In [ ]:
employement = jobs[jobs['employement_type '].notnull()]

employement['employement_type '].value_counts().to_frame()


In [ ]:
plt.figure(figsize=(10,5))
plt.tight_layout()
employement['employement_type '].value_counts().nlargest(15).plot(kind='bar')


In [ ]:
experience_Level = employement = jobs[jobs['experience_Level'].notnull()]

employement['experience_Level'].value_counts().to_frame()

In [ ]:
plt.figure(figsize=(10,5))
plt.tight_layout()
employement['experience_Level'].value_counts().nlargest(15).plot(kind='bar')


In [ ]:
discription_df = jobs[['job_description']].copy()
discription_df

In [ ]:
discription_df.isna().sum()

In [ ]:
rand_indexs = np.random.randint(1,len(discription_df),50).tolist()
discription_df["job_description"][rand_indexs]

In [ ]:
import string
def remove_punct(text):
    text  = "".join([char for char in text if char not in string.punctuation])
    text = re.sub('[0-9]+', '', text)
    text = re.sub(r'[^a-zA-Z0-9 \n\.]', ' ' , text)
    text = text.replace('\n', ' ')
    return text

discription_df['description_punct'] = discription_df["job_description"].apply(lambda x: remove_punct(x))
discription_df.head(10)

In [ ]:
def tokenization(text):
    text = re.split('\W+', text)
    return text

discription_df['discription_tokenized'] = discription_df['description_punct'].apply(lambda x: tokenization(x.lower()))
discription_df.head()

In [ ]:
stopword = nltk.corpus.stopwords.words('english')
def remove_stopwords(text):
    text = [word for word in text if word not in stopword]
    return text
    
discription_df['discription_nonstop'] = discription_df['discription_tokenized'].apply(lambda x: remove_stopwords(x))
discription_df.head(10)

In [ ]:
ps = nltk.PorterStemmer()

def stemming(text):
    text = [ps.stem(word) for word in text]
    return text

discription_df['discription_stemmed'] = discription_df['discription_nonstop'].apply(lambda x: stemming(x))
discription_df.head()

In [ ]:
wn = nltk.WordNetLemmatizer()

def lemmatizer(text):
    text = [wn.lemmatize(word) for word in text]
    return text

discription_df['discription_lemmatized'] = discription_df['discription_nonstop'].apply(lambda x: lemmatizer(x))
discription_df.head()

In [ ]:
from nltk.tokenize import word_tokenize

# Uncomment this line if you haven't downloaded punkt before
# or just run it as it is and uncomment it if you got an error.
# nltk.download('punkt')

def most_used_words(text):
    tokens = word_tokenize(text)
    frequency_dist = nltk.FreqDist(tokens)
    print("There is %d different words" % len(set(text)))
    return sorted(frequency_dist,key=frequency_dist.__getitem__, reverse=True)

In [ ]:
words = set(nltk.corpus.words.words())

test_text_data = " ".join(w for w in nltk.wordpunct_tokenize(discription_df.job_description.str.cat()) \
         if w.lower() in words or not w.isalpha())


In [ ]:
most_used_words(test_text_data)[:10]

In [ ]:
mw = most_used_words(discription_df.job_description.str.cat())
most_words = []
for w in mw:
    if len(most_words) == 1000:
        break
    if w in stopwords.words("english"):
        continue
    else:
        most_words.append(w)

In [ ]:
sorted(most_words)

In [ ]:
# I'm defining this function to use it in the 
# Data Preparation Phase
from nltk.stem.snowball import SnowballStemmer
from nltk.stem import WordNetLemmatizer

# nltk.download('wordnet')
def stem_tokenize(text):
    stemmer = SnowballStemmer("english")
    stemmer = WordNetLemmatizer()
    return [stemmer.lemmatize(token) for token in word_tokenize(text)]

def lemmatize_tokenize(text):
    lemmatizer = WordNetLemmatizer()
    return [lemmatizer.lemmatize(token) for token in word_tokenize(text)]

In [ ]:
from PIL import Image
from wordcloud import WordCloud, STOPWORDS, ImageColorGenerator

import matplotlib.pyplot as plt
# Start with one review:
text = " ".join(discription for discription in discription_df.description_punct)
# lower max_font_size, change the maximum number of word and lighten the background:
stopwords = set(STOPWORDS)
# stopwords.add(search_modifier)
wordcloud = WordCloud(max_font_size=50, max_words=100,stopwords= stopwords ,  background_color="white").generate(text)
wordcloud.to_file('N.png')
plt.figure(figsize=(100,50))
plt.tight_layout()
plt.imshow(wordcloud, interpolation="bilinear")
plt.axis("off")
plt.show()
# plt.savefig('wordcloud.png')

## Frequency Analysis

In [ ]:
#download nltk attribute if needed
import nltk
import ssl

try:
    _create_unverified_https_context = ssl._create_unverified_context
except AttributeError:
    pass
else:
    ssl._create_default_https_context = _create_unverified_https_context

nltk.download('reuters')

In [ ]:
text = ' '.join(description for description in discription_df.description_punct)

In [ ]:
text

In [ ]:
# tokenization
filtered_text = [word for sent in nltk.sent_tokenize(text) for word in nltk.word_tokenize(sent)]

In [ ]:
from nltk.corpus import stopwords
# stopwords
filtered_text = [word for word in filtered_text if word not in stopwords.words('english')]

In [ ]:
# punctuation
filtered_text = [''.join(c for c in s if c not in string.punctuation) for s in filtered_text]
filtered_text = [s for s in filtered_text if s]

In [ ]:
# lowercase
filtered_text = [word.lower() for word in filtered_text]

In [ ]:
# Word Stemming
from nltk.stem.snowball import SnowballStemmer

stemmer = SnowballStemmer("english")
filtered_text_stem = [stemmer.stem(t) for t in filtered_text]

In [ ]:
# Lemmatization
lemmatizer = nltk.WordNetLemmatizer()

filtered_text_lem = [lemmatizer.lemmatize(t) for t in filtered_text]

In [ ]:
# NLTK Frequency Distributions
#nltkText = [filtered_text_stem, filtered_text_lem]

fdist = nltk.FreqDist(filtered_text_lem)

fdistDF = pd.DataFrame.from_dict(fdist, orient='index').reset_index()
fdistDF = fdistDF.rename(columns={'index':'word', 0:'count'})
fdistDF = fdistDF.sort_values(by='count', ascending=False)
print('-----------------head words-----------------')
print(fdistDF.head(10).reset_index(drop=True))
print('-----------------tail words-----------------')
print(fdistDF.tail(10).reset_index(drop=True))

## Analyze document words with data science job descripiton database
To indentify what skills are specific for this job vacancy in particulair we have to create a data science job description database. Using this database we can identify what is specific about this job with the steps below.

Building a Term-Document Matrix
Use scikit-learn's TfidfVectorizer class to construct a term-document matrix containing the TF-IDF score for each word in each document in the data science job description database. In essence, the rows of this sparse matrix correspond to documents in the corpus, the columns represent each word in the vocabulary of the corpus, and each cell contains the TF-IDF value for a given word in a given document.

TF-IDF Scores
Now that we've built the term-document matrix, we can explore its contents:

In [ ]:
from random import randint
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.feature_extraction.text import CountVectorizer


# rand_indexs = np.random.randint(1,len(discription_df),1)
rand_indexs = np.random.randint(1,len(discription_df),1)
text = " ".join(discription_df["description_punct"][rand_indexs].tolist())
print(text)

tfidf = TfidfVectorizer(stop_words='english', analyzer='word', lowercase=True)
content = text.split()
tfidf_vector = tfidf.fit_transform(content)

feature_names = tfidf.get_feature_names()

vectorizer = CountVectorizer()
X = vectorizer.fit_transform(content)

tdm= pd.DataFrame(X.toarray(), columns=vectorizer.get_feature_names())
print ('TDM contains ' + str(len(feature_names)) + ' terms and ' + str(tdm.shape[0]) + ' documents')

print ('first term: ' + feature_names[0])
print ('last term: ' + feature_names[len(feature_names) - 1])

for i in range(0, 4):
    print ('random term: ' + feature_names[randint(1,len(feature_names) - 2)])

In [ ]:
discription_df.columns

### Generating the Summary
That's all we'll need to produce a summary for any document in the corpus. In the example code below, we start by randomly selecting an article from the data science job description database. We iterate through the article, calculating a score for each sentence by summing the TF-IDF values for each word appearing in the sentence. We normalize the sentence scores by dividing by the number of tokens in the sentence (to avoid bias in favor of longer sentences). Then we sort the sentences by their scores, and return the highest-scoring sentences as our summary. The number of sentences returned corresponds to roughly 20% of the overall length of the article.

In [ ]:
from __future__ import division
import math

def tokenize_and_stem(sentence):
    filtered_text = [word for sent in nltk.sent_tokenize(sentence) for word in nltk.word_tokenize(sent)]
    stemmer = SnowballStemmer("english")
    filtered_text_stem = [stemmer.stem(t) for t in filtered_text]
    return filtered_text_stem
    
    
    
article_id = randint(0, tdm.shape[0] - 1)
article_text = discription_df.iloc[article_id]
print(article_text)

sent_scores = []
for sentence in nltk.sent_tokenize(article_text):
    score = 0
    sent_tokens = tokenize_and_stem(sentence)
    for token in (t for t in sent_tokens if t in feature_names):
        score += tdm[article_id, feature_names.index(token)]
    sent_scores.append((score / len(sent_tokens), sentence))

summary_length = int(math.ceil(len(sent_scores) / 5))
sent_scores.sort(key=lambda sent: sent[0], reverse=True)

print ('*** SUMMARY ***')
for summary_sentence in sent_scores[:summary_length]:
    print (summary_sentence[1])

print ('\n*** ORIGINAL ***')
print (article_text)